<a href="https://colab.research.google.com/github/hany606/COEX-Internship19/blob/master/projects/Human_pose_estimation_drone_control/Faild_trials/Human_pose_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

######################################## **Colab environment** ########################################

---


In [0]:
# Connect with Google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


############################################ **Installation** ############################################

---



### Installation of required libraries: Openpose and its dependency



In [0]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  print("In process")
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  print("################################## Install CMake ##################################")
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  print("################################## Clone openpose ##################################")
  !git clone -q --depth 1 $git_repo_url
  #!cat openpose/CMakeLists.txt
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  !sed -i 's/"Build OpenPose python." OFF/"Build OpenPose python." ON/g' openpose/CMakeLists.txt
  #!cat openpose/CMakeLists.txt
  # install system dependencies
  print("################################## Install Dependencies ##################################")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # build openpose
  print("################################## Build openpose ##################################")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  print("################################## Finished Finished ##################################")
  

### Installation of required libraries: aiortc, aiohttp and its dependency

In [0]:
!apt install libavdevice-dev libavfilter-dev libopus-dev libvpx-dev pkg-config
!pip install aiortc aiohttp

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu2).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  libavdevice-dev libavfilter-dev libopus-dev libpostproc-dev libvpx-dev
0 upgraded, 5 newly installed, 0 to remove and 4 not upgraded.
Need to get 2,285 kB of archives.
After this operation, 9,738 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libpostproc-dev amd64 7:3.4.6-0ubuntu0.18.04.1 [51.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libavfilter-dev amd64 7:3.4.6-0ubuntu0.18.04.1 [1,016 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libavdevice-dev amd64 7:3.4.6-0ubuntu0.18.04.1 [87.2 kB]
Get:4 http://archive.ubuntu.com/ubuntu bioni

######################################## **Environment Setup** #######################################

---



Go to the directory of Python API where you can use it only there

In [0]:
!pwd
!ls
import os
os.chdir('/content/openpose/build/examples/tutorial_api_python')
!pwd

/content
cmake-3.13.0-Linux-x86_64.tar.gz  drive  openpose  sample_data
/content/openpose/build/examples/tutorial_api_python


Clone my repo for testing and debugging

In [0]:
!git clone https://github.com/hany606/COEX-Internship19.git

Cloning into 'COEX-Internship19'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 152 (delta 60), reused 110 (delta 37), pack-reused 0
Receiving objects: 100% (152/152), 758.25 KiB | 1.00 MiB/s, done.
Resolving deltas: 100% (60/60), done.


############################################ **Testing** ############################################

---



To enusre that the python API has been installed correctly, the result of the below cell should have one of the files: pyopenpose.cpython-36m-x86_64-linux-gnu.so

In [0]:
!ls ../../python/openpose
#pyopenpose.cpython-36m-x86_64-linux-gnu.so  should appear to be correctly installed for python3

CMakeFiles	     __init__.py  pyopenpose.cpython-36m-x86_64-linux-gnu.so
cmake_install.cmake  Makefile


Run an example:

In [0]:
!python3 01_body_from_image.py	

Modified version from the example to work with colab to show the images as colab doesn't have a x server to show the images as in normal OS.

In [0]:
# From Python
# It requires OpenCV installed for Python
import sys
import cv2
import os
from sys import platform
import argparse
from google.colab.patches import cv2_imshow

image_path = "../../../../drive/My Drive/frame.jpg"
#"../../../examples/media/COCO_val2014_000000000192.jpg"

# Import Openpose (Linux [Colab])
try:
    # Change these variables to point to the correct folder (Release/x64 etc.) 
    sys.path.append('../../python');
    # If you run `make install` (default path is `/usr/local/python` for Ubuntu), you can also access the OpenPose/python module from there. This will install OpenPose and the python library at your desired installation path. Ensure that this is in your python path in order to use it.
    # sys.path.append('/usr/local/python')
    from openpose import pyopenpose as op
except ImportError as e:
    print('Error: OpenPose library could not be found. Did you enable `BUILD_PYTHON` in CMake and have this Python script in the right folder?')
    raise e

# Flags
parser = argparse.ArgumentParser()
parser.add_argument("--image_path", default=image_path, help="Process an image. Read all standard formats (jpg, png, bmp, etc.).")
args = parser.parse_known_args()

# Custom Params (refer to include/openpose/flags.hpp for more parameters)
params = dict()
params["model_folder"] = "../../../models/"

# Add others in path?
for i in range(0, len(args[1])):
    curr_item = args[1][i]
    if i != len(args[1])-1: next_item = args[1][i+1]
    else: next_item = "1"
    if "--" in curr_item and "--" in next_item:
        key = curr_item.replace('-','')
        if key not in params:  params[key] = "1"
    elif "--" in curr_item and "--" not in next_item:
        key = curr_item.replace('-','')
        if key not in params: params[key] = next_item

# Construct it from system arguments
# op.init_argv(args[1])
# oppython = op.OpenposePython()

try:
    # Starting OpenPose
    opWrapper = op.WrapperPython()
    opWrapper.configure(params)
    opWrapper.start()

    # Process Image
    datum = op.Datum()
    imageToProcess = cv2.imread(args[0].image_path)
    datum.cvInputData = imageToProcess
    opWrapper.emplaceAndPop([datum])

    # Display Image
#     print("Body keypoints: \n" + str(datum.poseKeypoints))
    cv2_imshow(datum.cvOutputData)
#     cv2.waitKey(0)
except Exception as e:
    # print(e)
    sys.exit(-1)


Error: OpenPose library could not be found. Did you enable `BUILD_PYTHON` in CMake and have this Python script in the right folder?


ImportError: ignored

########################################## **Testing playground** ##########################################

---



######################################### **Openpose module** #########################################

---


########################################## **Camera module** ##########################################

---



##################################### **Video Stream with Openpose** #####################################

---



############################################ **Main code** ############################################

---



############################################ **Running area** ############################################

---



In [0]:
!pwd
!cd COEX-Internship19/projects/Human_pose_estimation/ && ls && pwd
#!python3 COEX-Internship19/projects/Human_pose_estimation/webcam/webcam.py

/content/openpose/build/examples/tutorial_api_python
Human_pose_estimation.ipynb  webcam
/content/openpose/build/examples/tutorial_api_python/COEX-Internship19/projects/Human_pose_estimation


In [0]:

import argparse
import asyncio
import json
import logging
import os
import platform
import ssl
import portpicker

from aiohttp import web
import cv2
from av import VideoFrame

from aiortc import RTCPeerConnection, RTCSessionDescription, VideoStreamTrack
from aiortc.contrib.media import MediaPlayer



from IPython.display import display, Javascript
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow

import time

# ROOT = os.path.dirname(__file__)
ROOT = "/content/COEX-Internship19/projects/Human_pose_estimation/webcam"
#openpose/build/examples/tutorial_api_python/

def process(frame):
    return frame
class VideoTransformTrack(VideoStreamTrack):
    def __init__(self, track):
        super().__init__()  # don't forget this!
        self.track = track

    async def recv(self):
        frame = await self.track.recv()
        print("Recev")
        # perform edge detection
        img = frame.to_ndarray(format="bgr24")
        img = process(img)
        cv2_imshow(img)
        # rebuild a VideoFrame, preserving timing information
        new_frame = VideoFrame.from_ndarray(img, format="bgr24")
        new_frame.pts = frame.pts
        new_frame.time_base = frame.time_base
        return new_frame

async def index(request):
    content = open(os.path.join(ROOT, "index.html"), "r").read()
    return web.Response(content_type="text/html", text=content)


async def javascript(request):
    content = open(os.path.join(ROOT, "client.js"), "r").read()
    return web.Response(content_type="application/javascript", text=content)

    
async def offer(request):
    params = await request.json()
    offer = RTCSessionDescription(sdp=params["sdp"], type=params["type"])

    pc = RTCPeerConnection()
    pcs.add(pc)

    @pc.on("iceconnectionstatechange")
    async def on_iceconnectionstatechange():
        print("ICE connection state is %s" % pc.iceConnectionState)
        if pc.iceConnectionState == "failed":
            await pc.close()
            pcs.discard(pc)

    # open webcam
    options = {"framerate": "30", "video_size": "640x480"}
    if platform.system() == "Darwin":
        player = MediaPlayer("default:none", format="avfoundation", options=options)
    else:
        player = MediaPlayer("/dev/video0", format="v4l2", options=options)
    
    await pc.setRemoteDescription(offer)
    for t in pc.getTransceivers():
        if t.kind == "audio" and player.audio:
            pc.addTrack(player.audio)
        elif t.kind == "video" and player.video:
            local_video = VideoTransformTrack(player.video)
            pc.addTrack(local_video)

    answer = await pc.createAnswer()
    await pc.setLocalDescription(answer)

    return web.Response(
        content_type="application/json",
        text=json.dumps(
            {"sdp": pc.localDescription.sdp, "type": pc.localDescription.type}
        ),
    )


pcs = set()

async def tryMe():
  time.sleep(5)
  print("POO hhh")

async def on_shutdown(app):
    # close peer connections
    coros = [pc.close() for pc in pcs]
    await asyncio.gather(*coros)
    pcs.clear()


if __name__ == "__main__":
    ssl_context = None

#     app = web.Application()
#     app.on_shutdown.append(on_shutdown)
#     app.router.add_get("/", index)
#     app.router.add_get("/client.js", javascript)
#     app.router.add_post("/offer", offer)
    port = portpicker.pick_unused_port()
    content = Javascript('''
    var pc = new RTCPeerConnection({
    sdpSemantics: 'unified-plan'
    });

    // connect audio / video
    pc.addEventListener('track', function(evt) {
        if (evt.track.kind == 'video') {
            // document.getElementById('video').srcObject = evt.streams[0];
        } else {
            // document.getElementById('audio').srcObject = evt.streams[0];
        }
    });

    function negotiate() {
        pc.addTransceiver('video', {direction: 'recvonly'});
        pc.addTransceiver('audio', {direction: 'recvonly'});
        console.log("negot")
        return pc.createOffer().then(function(offer) {
            return pc.setLocalDescription(offer);
        }).then(function() {
            // wait for ICE gathering to complete
            return new Promise(function(resolve) {
                if (pc.iceGatheringState === 'complete') {
                    console.log("Then")
                    resolve();
                } else {
                    function checkState() {
                        if (pc.iceGatheringState === 'complete') {
                            pc.removeEventListener('icegatheringstatechange', checkState);
                            resolve();
                        }
                    }
                    pc.addEventListener('icegatheringstatechange', checkState);
                }
            });
        }).then(function() {
        
            var offer = pc.localDescription;
            return fetch('/offer', {
                body: JSON.stringify({
                    sdp: offer.sdp,
                    type: offer.type,
                }),
                headers: {
                    'Content-Type': 'application/json'
                },
                method: 'POST'
            });
        }).then(function(response) {
            res  = response.json();
            console.log(res);
            return res;
        }).then(function(answer) {
            console.log("sada")
            return pc.setRemoteDescription(answer);
        }).catch(function(e) {
            alert(e);
        });
    }

    function start() {
        console.log('START')
        negotiate();
    }

    function stop() {
        // close peer connection
        setTimeout(function() {
            pc.close();
        }, 500);
    }
  ''')
    display(content)
#     data = eval_js('start()')
#     port = 20248
    print("Runnign on port: {:}".format(port))
    
#     while(1):
#       time.sleep(5)
#     web.run_app(app, host= '0.0.0.0',port= port, ssl_context=ssl_context)
    print("asd")

<IPython.core.display.Javascript object>

Runnign on port: 21517


RuntimeError: ignored

In [0]:
%%px0
import requests as req
url = 'http://127.0.0.1:18744'
resp = req.get(url)
print(resp.text)

UsageError: Cell magic `%%px0` not found.


In [0]:

!pip install ipyparallel


     |████████████████████████████████| 204kB 2.7MB/s 


In [0]:
import ipyparallel as ipp
rc = ipp.Client()

Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-client.json


OSError: ignored

In [0]:
%%px
print("Hello")

In [0]:
!rm -rf COEX-Internship19/

In [0]:
!cd COEX-Internship19/ && git pull

remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 16 (delta 11), reused 14 (delta 9), pack-reused 0
Unpacking objects: 100% (16/16), done.
From https://github.com/hany606/COEX-Internship19
   04f2e18..38190e7  master     -> origin/master
Updating 04f2e18..38190e7
Fast-forward
 projects/Human_pose_estimation/webcam/client.js    |   1 +
 projects/Human_pose_estimation/webcam/client2.js   |  66 ++++++++++++
 projects/Human_pose_estimation/webcam/index.html   |   2 +-
 projects/Human_pose_estimation/webcam/webcam.py    |   2 +
 .../Human_pose_estimation/webcam/webcam_colab.py   | 120 +++++++++++++++++++++
 5 files changed, 190 insertions(+), 1 deletion(-)
 create mode 100644 projects/Human_pose_estimation/webcam/client2.js
 create mode 100644 projects/Human_pose_estimation/webcam/webcam_colab.py


In [0]:
import argparse
import asyncio
import logging
import math

import cv2
import numpy
from av import VideoFrame

from aiortc import (
    RTCIceCandidate,
    RTCPeerConnection,
    RTCSessionDescription,
    VideoStreamTrack,
)
from aiortc.contrib.media import MediaBlackhole, MediaPlayer, MediaRecorder
from aiortc.contrib.signaling import add_signaling_arguments, create_signaling


class FlagVideoStreamTrack(VideoStreamTrack):
    """
    A video track that returns an animated flag.
    """

    def __init__(self):
        super().__init__()  # don't forget this!
        self.counter = 0
        height, width = 480, 640

        # generate flag
        data_bgr = numpy.hstack(
            [
                self._create_rectangle(
                    width=213, height=480, color=(255, 0, 0)
                ),  # blue
                self._create_rectangle(
                    width=214, height=480, color=(255, 255, 255)
                ),  # white
                self._create_rectangle(width=213, height=480, color=(0, 0, 255)),  # red
            ]
        )

        # shrink and center it
        M = numpy.float32([[0.5, 0, width / 4], [0, 0.5, height / 4]])
        data_bgr = cv2.warpAffine(data_bgr, M, (width, height))

        # compute animation
        omega = 2 * math.pi / height
        id_x = numpy.tile(numpy.array(range(width), dtype=numpy.float32), (height, 1))
        id_y = numpy.tile(
            numpy.array(range(height), dtype=numpy.float32), (width, 1)
        ).transpose()

        self.frames = []
        for k in range(30):
            phase = 2 * k * math.pi / 30
            map_x = id_x + 10 * numpy.cos(omega * id_x + phase)
            map_y = id_y + 10 * numpy.sin(omega * id_x + phase)
            self.frames.append(
                VideoFrame.from_ndarray(
                    cv2.remap(data_bgr, map_x, map_y, cv2.INTER_LINEAR), format="bgr24"
                )
            )

    async def recv(self):
        pts, time_base = await self.next_timestamp()

        frame = self.frames[self.counter % 30]
        frame.pts = pts
        frame.time_base = time_base
        self.counter += 1
        return frame

    def _create_rectangle(self, width, height, color):
        data_bgr = numpy.zeros((height, width, 3), numpy.uint8)
        data_bgr[:, :] = color
        return data_bgr


async def run(pc, player, recorder, signaling, role):
    def add_tracks():
        if player and player.audio:
            pc.addTrack(player.audio)

        if player and player.video:
            pc.addTrack(player.video)
        else:
            pc.addTrack(FlagVideoStreamTrack())

    @pc.on("track")
    def on_track(track):
        print("Receiving %s" % track.kind)
        recorder.addTrack(track)

    # connect signaling
    await signaling.connect()

    if role == "offer":
        # send offer
        add_tracks()
        await pc.setLocalDescription(await pc.createOffer())
        await signaling.send(pc.localDescription)

    # consume signaling
    while True:
        obj = await signaling.receive()

        if isinstance(obj, RTCSessionDescription):
            await pc.setRemoteDescription(obj)
            await recorder.start()

            if obj.type == "offer":
                # send answer
                add_tracks()
                await pc.setLocalDescription(await pc.createAnswer())
                await signaling.send(pc.localDescription)
        elif isinstance(obj, RTCIceCandidate):
            pc.addIceCandidate(obj)
        elif obj is None:
            print("Exiting")
            break

# create signaling and peer connection
signaling = create_signaling()
pc = RTCPeerConnection()


recorder = MediaRecorder("f.avi")

# run event loop
loop = asyncio.get_event_loop()
loop.run_until_complete(
    run(
        pc=pc,
        player=player,
        recorder=recorder,
        signaling=signaling,
#         role=args.role,
    )
)

# cleanup
loop.run_until_complete(recorder.stop())
loop.run_until_complete(signaling.close())
loop.run_until_complete(pc.close())


TypeError: ignored

In [0]:
!google-chrome www.windows-commandline.com


/bin/bash: google-chrome: command not found
